# FIT5196 Assessment 2
# FIT5196 Task  in Assessment 2
#### Student Name: Pattranit Chaiyabud and Viet Tai Le
#### Student ID: 30304148 and 29975336 

Date: 14/09/2019

Version: 1.0

Environment: Python 3.6.0 and Anaconda 4.3.0 (64-bit)

Libraries used:
* re (for regular expression, included in Anaconda Python 3.6.0)
* os (for executing in subshell, included in Anaconda Python 3.6.0)
* urllib (for for retrive the pdf from url, include in Anaconda Python 3.6.0)
* itertools ( for iteration, include in Anaconda Python 3.6.0 )
* nltk 3.2.2 (Natural Language Toolkit, included in Anaconda Python 3.6)
* nltk.collocations (for finding bigrams, included in Anaconda Python 3.6)
* nltk.tokenize (for tokenization, included in Anaconda Python 3.6)
* nltk.probability (for frequency, included in Anaconda Python 3.6)
* nltk.stem (for PorterStemmer, included in Anaconda Python 3.6)
* nltk.corpus (for stopword, included in Anaconda Python 3.6)
* pandas (for dataframe, included in Anaconda Python 3.6.0) 



## 1. Introduction
This assignment found on the execution of different analyzing textual data such as extracting data from non-structured format and converting into the proper format suitable for a downstream modelling task. A data-set containing 200 URLs for paper published in a popular AI conference is given in the file named `Group148.pdf`. The required tasks are the following:

Extracting 200 URLs from the given pdf-file then use the extracted URLs to programmatically download the PDF files and read into text. After that extract the required entities to complete the task below:
1. Generate a sparse representation for `Paper Bodies` consisting of two of the following files
    - Vocabulary index file
    - Sparse count vectors file
2. Generate a CSV file containing `top most frequent word` for three of the following columns
    - Titles
    - Authors
    - Abstracts
    
More details for each task will be given in the following sections.
## 2.  Import libraries 

In [5]:
import re
import os
import nltk
from nltk.collocations import *
from itertools import chain
import itertools
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import MWETokenizer
from nltk.tokenize import sent_tokenize 
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.probability import *
import urllib
import pandas as pd

## 3. Examining File  and  prepare Data
As a first step, we need to use `pdfminer package` to convert pdf file `Group148.pdf` to txt file `Group148.txt`

In [6]:
# convert pdf file to txt file
!pdf2txt.py -o Group148.txt Group148.pdf

Traceback (most recent call last):
  File "C:\Users\mio\Anaconda3\Scripts\pdf2txt.py", line 136, in <module>
    if __name__ == '__main__': sys.exit(main())
  File "C:\Users\mio\Anaconda3\Scripts\pdf2txt.py", line 131, in main
    outfp = extract_text(**vars(A))
  File "C:\Users\mio\Anaconda3\Scripts\pdf2txt.py", line 62, in extract_text
    with open(fname, "rb") as fp:
FileNotFoundError: [Errno 2] No such file or directory: 'Group148.pdf'


Let see how all URLs in text file look like

In [7]:
# this is how all URLs look like
pdf_txt = open('./Group148.txt', 'r')
# loop over all the lines
for line in pdf_txt:
    # repr() is a built-in Python fuction that returns a string containing a printable representation of an object.
    print (repr(line))

The above code read the text file line-by-line and printed each line. You should notice that we have converted each line into a printable representation of a string object using Python's build-in function, `repr()`, as it will help us discover some patterns that can be used to extract those URLs.

Next we will use `re package` to extract the URLs from the pattern above and put them all together in a dictionary before further process.

In [8]:
#open file and read line by line
pdfTxtFile = './Group148.txt'
with open(pdfTxtFile, 'r') as f:
    pdf_txt=f.readlines()   
# Create a dictionary contains pdf name and pdf links 
list_pdf=[]
list_name=[]
#spot the pattern of both name and URLs
reg = re.compile(r'(https:.*)')
regpdf=re.compile(r'(PP.*?)\.')
#extract and add URLs to the list
for line in pdf_txt:
    if reg.search(line):
        list_pdf.append((reg.search(line).group(1)))
#extract and add name to the list
for line in pdf_txt:
    if regpdf.search(line):
        list_name.append((regpdf.search(line).group(1)))
#combine both name and URLs together as key-value dictionary
dict1=dict(zip(list_name,list_pdf))

After arranging the file in dictionary, we programmatically download the PDF files from the given URLs using `urlib.request` then read the PDF files into text. Now we are ready for all the task! 

Note that this step may a bit of time to execute because we have 200 files to perform. 

In [16]:
# Download all pdf files using urllib.request
for key,value in dict1.items():
    urllib.request.urlretrieve (value, key+ ".pdf")
#Convert pdf files to txt files
for key in dict1.keys():
    os.system("pdf2txt.py -o %s %s" % (key+'.txt', key+'.pdf'))

# 4. Task 1

Before building the sparse representation, you will need to perform text preprocessing on Paper Bodies. Please follow the below text preprocessing steps (not necessarily in the same order, you will need to figure out the correct order of operations that produces the correct set of vocabulary) 

A. The word tokenization must use the following regular expression, r"[A-Za-z]\w+(?:['?]\w+)?" 

B. The context-independent and context-dependent (with the threshold set to %95) stop words must be removed from the vocab. The context-independent stop words list (i.e, stopwords_en.txt) provided in the zip file must be used.

C. Unigram tokens should be stemmed using the Porter stemmer. (be careful that stemming performs lower casing by default)

D. Rare tokens (with the threshold set to 3%) must be removed from the vocab. 

E. Tokens must be normalized to lowercase except the capital tokens appearing in the middle of a sentence/line. (use sentence segmentation to achieve this) 

F. Tokens with the length less than 3 should be removed from the vocab.  

G. First 200 meaningful bigrams  (i.e., collocations), based on highest total frequency in the corpus, must be extracted and included in your tokenization process. Bigrams should not include context-independent stopwords as part of them and they should be separated using double underscore i.e. “__”  (example: “artifical____intelligence”) 

Firstly, we need to extract content of Paper body section by using re.search with patern `r'Paper Body(.*?)2 References'`. After that, we create a dictionary that contains document name and its paper body correspondently. 

In [17]:
# Task 1
# Preparing data
rawdict={}
filetxt=[key+'.txt' for key in dict1.keys()]
for file in filetxt:
    rawdoc=''
    with open(file,'r',encoding='utf-8') as f:
        for line in f:
            rawdoc += line.strip()
        rawdata=re.search(r'Paper Body(.*?)2 References',rawdoc).group(1)
        rawdict[file[:-4]]=rawdata
        

<b>In order to implement task 1</b>, the order of text preprocessing steps is E,A,B,D,F,C. At first, i combined step E and A by define a function name <b>tokenize</b> which will process sentence Segmentation and lowercase the first word of each segment. After that, the function will tokenize all segment in each document. 

In the next step, after using for loop to tokenize all documents in corpus, the method chain.from_iterable can help to concat all document's tokens to a list of tokens for a whole corpus 


In [19]:
# sentence segmentation,tokenizer
from nltk.tokenize import sent_tokenize

def doc_tokenize(doc):
    segdoc=sent_tokenize(rawdict[doc])
    tokenizer = RegexpTokenizer(r"[A-Za-z]\w+(?:['?]\w+)?")
    tokens=[]
    for sentence in segdoc:
        list1=sentence.strip().split(' ')
        list1[0]=list1[0].lower()
        sentence=' '.join(list1)
        tokens+=tokenizer.tokenize(sentence)
    return (doc,tokens)

doc_tokenized = dict(doc_tokenize(doc) for doc in rawdict.keys())
all_words = list(chain.from_iterable(doc_tokenized.values()))
doc_tokenized

{}

The next stage is creating top 200 bigram based on highest frequency in the corpus. In order to do that, we need *BigramAssocMeasures() 
*BigramCollocationFinder.from_words()
*bigram_finder.apply_word_filter(lambda w: w in stopindewords)
*nbest(bigram_measures.raw_freq, 200)

To introduce top 200 bigrams to tokens, a method MWETokenizer which takes a string which has already been divided into tokens and retokenizes it, merging multi-word expressions into single tokens.

In [9]:
# create stopindewords
with open('stopwords_en.txt','r') as f:
    stopindewords=f.read().splitlines()
# bigram
bigram_measures = nltk.collocations.BigramAssocMeasures()
bigram_finder = nltk.collocations.BigramCollocationFinder.from_words(all_words)
bigram_finder.apply_word_filter(lambda w: w in stopindewords)
# raw_freq to find top 200 based on high frequency 
top_200_bigrams = bigram_finder.nbest(bigram_measures.raw_freq, 200) 
# alternatively, can use ngram_fd.items() to find top 200
# put bigram to tokens
mwetokenizer = MWETokenizer(top_200_bigrams,separator='__')
colloc_doc =  dict((key, mwetokenizer.tokenize(value)) for key,value in doc_tokenized.items())
all_words_colloc = list(chain.from_iterable(colloc_doc.values()))

The next step here is create sets that contains context dependent stop words and rare tokens. 

In [10]:
# create list of stopword
words_1 = list(chain.from_iterable([set(value) for value in colloc_doc.values()]))
fd_1 = FreqDist(words_1)
# Threshold of rare tokens
min_doc=0.03*len(rawdict)

max_doc=0.95*len(rawdict)
# set of rare tokens
rare_tokens={token for token,value in fd_1.items() if value <= min_doc}
# set of context dependent stopwords
stopdewords={token for token,value in fd_1.items() if value >= max_doc }

Using set comprehension and list comprehension to remove all stopwords,rare tokens and token with length less than 3 from vocabulary

In [11]:
vocab=set(all_words_colloc)
# remove stopwords from vocab
token_filtered={token for token in vocab if token not in (stopindewords and stopdewords )}
# remove rare tokens from vocab
removed_rare_token=[token for token in token_filtered if token not in rare_tokens]
#remove token less 3 from vocab
removed3_tokens=[token for token in removed_rare_token if len(token)>=3]

Since by default stem method lower case all words, we have to re-define method without lower casing words.

In [12]:
# import Porter Stemmer and change default parameter of stem 
from nltk.stem import PorterStemmer

def stem(self, word):
        stem = word

        if self.mode == self.NLTK_EXTENSIONS and word in self.pool:
            return self.pool[word]

        if self.mode != self.ORIGINAL_ALGORITHM and len(word) <= 2:
            # With this line, strings of length 1 or 2 don't go through
            # the stemming process, although no mention is made of this
            # in the published algorithm.
            return word

        stem = self._step1a(stem)
        stem = self._step1b(stem)
        stem = self._step1c(stem)
        stem = self._step2(stem)
        stem = self._step3(stem)
        stem = self._step4(stem)
        stem = self._step5a(stem)
        stem = self._step5b(stem)

        return stem
PorterStemmer.stem = stem

Using stem method that we re-define above to stemming all unigram and after that remove all duplicate items in vocabulary

In [13]:
# Stemming token
stemmer = PorterStemmer()
vocab=[stemmer.stem(w) if '__' not in w else w for w in removed3_tokens]
# remove all duplicate words from vocab
vocab=sorted(set(vocab))

Export vocabulary into required format by creating a dictionary which keys are token and values are their index

In [14]:
#export vocab to txt file
vocab_output=''
vocab_dict={token[1]:token[0] for token in enumerate(vocab)}
for key,value in vocab_dict.items():
    vocab_output+=key +':'+ str(value) +' \n'
with open('Group148_vocab.txt','w',encoding='utf-8') as f:
    f.write(vocab_output)

To export count vector,firstly, we need to stemming all unigrams in each document's tokens. Secondly, we have to count term frequency of each words in a document by method FreqDist. After that, If else statement can help to filter token that in vocabulary and have frequency greater than 0.

In [15]:
#create count vector and export it to txt file
count_vector_corpus=''

for doc in colloc_doc.keys():
    # dictionary contains number of occurance of stemmed token in each document
    # stemming unigram token in each doc
    stemm_token=[]
    stemm_token=[stemmer.stem(token) if '__' not in token else token for token in colloc_doc[doc]]
    # counting orrcurance
    fd_token=FreqDist(stemm_token)
    # apply index of token in vocab to the above dictionary 
    count_vector={}
    for token,w_count in fd_token.items():
        if token in vocab_dict.keys() and w_count >0:
            count_vector[vocab_dict[token]]=w_count
    count_vector_corpus+=doc+','+str(count_vector).strip('{}').replace(' ','')+'\n'
with open('Group148_count_vectors.txt','w',encoding='utf-8') as f:
    f.write(count_vector_corpus)

# 5. Task 2

Before Statistics Generation for the second task we will need to perform the following preprocessing steps on the Titles and Abstracts before extracting the required stats:

A. The word tokenization must use the following regular expression, `r"[A-Za-z]\w+(?:[-'?]\w+)?"`

B. The context-independent stop words (i.e, stopwords_en.txt) must be removed

C. For Abstracts, Tokens must be normalized to lowercase except the capital tokensappearing in the middle of a sentence/line. (use sentence segmentation to achieve this). For Titles, tokens must be all normalised to lowercase.

Starting with preparing the data before extracting by make the data in Dictionary(key,value). While key is the name of the file and value is the whole content of the file.

In [16]:
#Task2
#preparing data
files ={}
filenames = [key+'.txt' for key in dict1.keys()]
for each in filenames:
    with open(each,'r', encoding="utf8") as f:
        if each in files:
            continue
        files[each] = f.read().strip()          

Next create 5 of the following functions to support each job of extraction and statistic generation.

Create the first function called `clean()` to remove all the newline using `str.strip()` and `re.sub` to get cleaner look of data 

In [17]:
def clean(value):
    clean_list = [x.strip() for x in value]
    clean_list = ''.join(clean_list)
    clean_list = re.sub(r'\n',' ',clean_list)
    clean_list = re.sub(r'\-\s','',clean_list)
    return clean_list

The second function created is `get_author()`. This function join each line after extracting the author name in the next few step and differentiate the author name by `\n` between names using `str.split()` then remove `''` from the list to get the whole list of authors' name.

In [18]:
def get_author(value):
    author_list = ''.join(value)
    author_list = author_list.splitlines()
    author_list = [x for x in author_list if x] 
    return author_list

Next two function is `get_title()` and `get_abstract()` function. After getting cleaned extracted data, we split the sentence into token by `[A-Za-z]\w+(?:[-?]\w+)?` for both of them and remove stop word at the end.

In [19]:
def get_title(value):
    title_list = clean(value)
    tokenizer = RegexpTokenizer(r'[A-Za-z]\w+(?:[-?]\w+)?')
    title_list = tokenizer.tokenize(title_list)
    title_list = [x.lower() for x in title_list]
    title_list = [x for x in title_list if x not in stopindewords]
    return title_list    

Before tokenization of `get_abstract()` function, we split sentence by `sent_tokenize()` since there might be capital word appearing in the middle of sentence/line which should not be lower cased as it would make them lose their meaning. Therefore, lowercase only the first letter of each sentence.

In [20]:
def get_abstract(value):
    abstract_list = clean(value)
    abstract_list = sent_tokenize(abstract_list)
    abstract_list = [x[0].lower() + x[1:] for x in abstract_list]
    abstract_list = ''.join(abstract_list)
    tokenizer = RegexpTokenizer(r'[A-Za-z]\w+(?:[-?]\w+)?')
    abstract_list = tokenizer.tokenize(abstract_list)  
    abstract_list = [x for x in abstract_list if x not in stopindewords]
    return abstract_list

The next function is `get_stat()` which calculate the frequency of word on the list and grab the most 10 frequent word.

Note that the list of the most frequent word is based on alphaetical ascending order in the case that the word appears as many time as the other.

In [21]:
def get_stat(value):
    stat = FreqDist(value).most_common(10)
    stat = sorted(stat, key = lambda x: (-x[1], x[0]))
    stat = [x[0] for x in stat]
    return stat

Now we are ready to perform the statistic generation!

In [22]:
#create list for title, author and abstract respectively
title_list = []
author_list = []
abstract_list = []
#regex to match the pattern for title, author and abstract
reg3 = re.compile(r'Authored\sby:([\s\S]*?)Abstract')
reg4 = re.compile(r'([\s\S]+?)Authored\sby')
reg5 = re.compile(r'Abstract([\s\S]+?)1\sPaper\sBody')
#insert matched pattern into those lists 
for key, value in files.items():
    if reg4.search(value):
        title_list.append(reg4.search(value).group(1))
    if reg3.search(value):
        author_list.append(reg3.search(value).group(1))
    if reg5.search(value):
        abstract_list.append(reg5.search(value).group(1))
#combine all the three list
combine_list = list(zip(get_stat(get_title(title_list)), \
                        get_stat(get_abstract(abstract_list)), get_stat(get_author(author_list)))) 

As you can see from above, we wrap the function get_title(), get_abstract() and get_author() for the list of words and finally use get_stat() to complete the task.
The result of combined list is shown below

In [23]:
combine_list

[('learning', 'data', 'Bernhard Sch?lkopf'),
 ('neural', 'learning', 'Lawrence Carin'),
 ('deep', 'model', 'Peter Dayan'),
 ('inference', 'problem', 'Piyush Rai'),
 ('gaussian', 'show', 'Quoc V. Le'),
 ('optimization', 'algorithm', 'Shivani Agarwal'),
 ('sparse', 'method', 'Zohar S. Karnin'),
 ('markov', 'models', 'Eric P. Xing'),
 ('analysis', 'approach', 'J?rgen Schmidhuber'),
 ('model', 'methods', 'Jun Zhu')]

Note that you may see the `?` in some of the name list as a result of pdf file that also have the `?` appear on some of the names.

In [24]:
#put the list in datafram and give the column name
df = pd.DataFrame(combine_list, columns= ['top10_terms_in_abstracts','top10_terms_in_titles','top10_authors'])
#export dataframe to csv file
df.to_csv('Group148_stats.csv', index=False)

# 6. Summary
This assessment measured the understanding of textual data analyzing techniques in the Python programming language. The main outcomes achieved while applying these techniques were:

- **Data extraction**. By using the built-in package called `re` or Regex module. With functions like `search()` and `sub()`, allow us to spot all the pattern of data with only a few inspections inluding replace unecessary characters in a proper way.
- **Pre-Processed text and Generating Features**. By using the `nltk` package, converting unstructured and semi-structured text into a structured representation before text analysis, then generate different vector representations and learn how to compute some basic statistics for text.

# 7.Reference

- garbage, D. (2019). Downloading a pdf file in python with urllib results in some html garbage. Retrieved 14 September 2019, from https://stackoverflow.com/questions/52273055/downloading-a-pdf-file-in-python-with-urllib-results-in-some-html-garbage
- PDF, R., Pietzcker, T., & Thoma, M. (2019). Read PDF in Python and convert to text in PDF. Retrieved 14 September 2019, from https://stackoverflow.com/questions/23821204/read-pdf-in-python-and-convert-to-text-in-pdf
- Dib, F. (2019). Online regex tester and debugger: PHP, PCRE, Python, Golang and JavaScript. Retrieved 22 August 2019, from https://regex101.com
- Exploring Pre-Processed text and Generating Features. (2019). .ipynb. (from moodle week5)
